# Test cache management

In [ ]:
import lamindb_setup as ln_setup
from lamindb_setup.dev._settings_store import system_storage_settings_file

ln_setup.login("testuser2")

In [ ]:
if system_storage_settings_file().exists():
    system_storage_settings_file().unlink()

In [ ]:
ln_setup.init(storage="s3://lndb-setup-ci", name="test-cache")

Check that setting the cache dir works.

In [ ]:
assert not system_storage_settings_file().exists()

In [ ]:
save_cache_dir = ln_setup.settings.storage.cache_dir
set_cache_dir = save_cache_dir.parent / "Cache1"

In [ ]:
ln_setup.settings.storage.cache_dir = set_cache_dir

In [ ]:
assert system_storage_settings_file().exists()

In [ ]:
assert ln_setup.settings.storage.cache_dir != save_cache_dir

In [ ]:
assert ln_setup.settings.storage.cache_dir == set_cache_dir

In [ ]:
assert set_cache_dir.exists()

In [ ]:
assert set_cache_dir in ln_setup.settings.instance._sqlite_file_local.parents

Check clearing the cache dir, also closes the cloud sqlite instance.

In [ ]:
!lamin cache clear

In [ ]:
assert not any(set_cache_dir.iterdir())

Check that the cache dir is preserved between`close` and `load`.

In [ ]:
ln_setup.load("test-cache")

In [ ]:
assert ln_setup.settings.storage.cache_dir == set_cache_dir

Setting to `"null"` or `None` restores the default directory.

In [ ]:
ln_setup.settings.storage.cache_dir = None

In [ ]:
assert ln_setup.settings.storage.cache_dir == save_cache_dir

In [ ]:
assert ln_setup.settings.storage.cache_dir != set_cache_dir

Setting the cache dir through CLI doesn't affect the currently loaded settings because it is done in another process and the settings only check the cache dir in `settings.env` on init.

Check CLI.

In [ ]:
!lamin cache

In [ ]:
!lamin cache set "null"

Clean up.

In [ ]:
system_storage_settings_file().unlink()
ln_setup.delete("test-cache", force=True)